## Peer-reviewed project for week 3 of Applied Data Science Capstone

Part 1 - Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe

In [2]:
#if necessary
#!pip install BeautifulSoup4


     |████████████████████████████████| 112kB 22.2MB/s eta 0:00:01


In [6]:
#if necessary
#!pip install lxml
#i'll say not necessary. having issue so I changed "lxml" to "html" and that appears to have worked.

     |████████████████████████████████| 5.8MB 23.5MB/s eta 0:00:01��███████▎  | 5.3MB 23.5MB/s eta 0:00:01


In [2]:
import pandas as pd
import numpy as np
import requests 
from bs4 import BeautifulSoup

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
url


'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [4]:
results = requests.get(url)
#results.content
table = BeautifulSoup(results.content,'html').find_all('table')[0]
#table

In [5]:
table_list = pd.read_html(str(table))
#table_list[0]

In [6]:
neighborhood = pd.DataFrame(table_list[0]) 

#The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
neighborhood.rename(columns={'Postal code': 'PostalCode'}, inplace=True)

#Only process the cells that have an assigned borough. 
#Ignore cells with a borough that is Not assigned.
#neighborhood.dropna(subset = ['Borough'])
neighborhood = neighborhood[neighborhood.Borough != 'Not assigned']

#If a cell has a borough but a Not assigned neighborhood, then the neighborhood 
#will be the same as the borough.
#note that removing the rows with missing Boroughs also resolves nulls in Neighborhood 
#but this command will remove them if any occur
neighborhood['Neighborhood'] = neighborhood['Neighborhood'].replace(np.nan, neighborhood['Borough'])
#neighborhood[neighborhood.Neighborhood.isnull()]

#More than one neighborhood can exist in one postal code area. 
#For example, in the table on the Wikipedia page, you will notice that 
#M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. 
#These two rows will be combined into one row with the neighborhoods separated 
#with a comma as shown in row 11 in the above table.
#neighborhood[neighborhood.PostalCode == 'M5A']
#neighborhood.duplicated('PostalCode')]
#I think duplicates have been resolved already

#neighborhood.head(10)

#In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
neighborhood.shape

(103, 3)

Part 2 - Use the Geocoder package or the csv file to get the latitude and the longitude coordinates of each neighborhood

In [7]:
#Start with a work copy of the data
df = neighborhood
df['Latitude'] = 0.0 
df['Longitude'] = 0.0 
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,0.0,0.0
3,M4A,North York,Victoria Village,0.0,0.0
4,M5A,Downtown Toronto,Regent Park / Harbourfront,0.0,0.0
5,M6A,North York,Lawrence Manor / Lawrence Heights,0.0,0.0
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,0.0,0.0


/#Geocoder was unusable for me. Even after installing an importing I was getting access issues. Maybe my syntax was off.
Eventually I went with the CSV, see next cell
!pip install geocoder

import geocoder # import geocoder

for index, row in df.iterrows(): postal_code = row['PostalCode']

/# initialize your variable to None
lat_lng_coords = None
check = 0 

/# loop until you get the coordinates
while(lat_lng_coords is None and check < 10):
    check = check + 1 
    /#g = geocoder.google('{}, Toronto, ON'.format(postal_code))
    g = geocoder.google('Toronto, ON')
    lat_lng_coords = g.latlng

print(postal_code)
postal_code = row['PostalCode']
print(g)
/#latitude = lat_lng_coords[0]
/#longitude = lat_lng_coords[1]
/#row['latitude'] = latitude 
/#row['longitude'] = longitude 

In [8]:
#Fetch the CSV
!wget -O Geospatial_data.csv http://cocl.us/Geospatial_data

--2020-04-15 20:21:46--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.48.113.194, 158.85.108.83, 158.85.108.86
Connecting to cocl.us (cocl.us)|169.48.113.194|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2020-04-15 20:21:46--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|169.48.113.194|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-04-15 20:21:49--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.27.197, 107.152.26.197
Connecting to ibm.box.com (ibm.box.com)|107.152.27.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-04-15 20:21:49--  https://

In [10]:
#read into dataframe
pcodes = pd.read_csv("Geospatial_data.csv")
pcodes.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
#update the work dataframe
for index, row in df.iterrows():
    postal_code = row['PostalCode']
    z = pcodes.index[pcodes['Postal Code'] == postal_code].tolist()
    latitude = pcodes.at[z[0], 'Latitude']
    longitude = pcodes.at[z[0], 'Longitude']
    #print(str(latitude) + ',' + str(longitude))
    df.at[index, 'Latitude'] = latitude
    df.at[index, 'Longitude'] = longitude

In [12]:
#Overwrite start dataframe with work
neighborhood = df
neighborhood.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
5,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


Step 3 - Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

In [13]:
#limit to boroughs with Toronto in name 
map_data = neighborhood[neighborhood.Borough.str.contains('toronto', case = False)]
map_data = map_data.drop(['PostalCode'], axis=1)
map_data = map_data.reset_index()
map_data.head()

,index,Borough,Neighborhood,Latitude,Longitude
0,4,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
1,6,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
2,13,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,22,Downtown Toronto,St. James Town,43.651494,-79.375418
4,30,East Toronto,The Beaches,43.676357,-79.293031


In [58]:
# @hidden_cell doesn't appear to work. I removed these values before publishing
# FourSquare creds
CLIENT_ID = '******' # your Foursquare ID
CLIENT_SECRET = '******' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ******
CLIENT_SECRET:******


In [22]:
# if necessary
#!conda install -c conda-forge geopy --yes 


Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          92 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.21.0-py_0



geopy-1.21.0         | 58 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [23]:
# if necessary
#!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    chardet-3.0.4              |py36h9f0ad1d_1006         188 KB  conda-forge
    cryptography-2.8           |   py36h45558ae_2         628 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    jinja2-2.11.2              |     pyh9f0ad1d_0          93 KB  conda-forge
    pandas-1.0.3               |   py36h830a2c2_0        11.1 MB  conda-forge
    pysocks-1.7.1              |   py36h9f0ad1d_1          27 KB  conda-

In [36]:
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [16]:
#Use geopy library to get the latitude and longitude values of Toronto.
#In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent ca_explorer, as shown below.
address = 'Toronto, ON'
geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [18]:
#Create a map of Toronto with neighborhoods superimposed on top.
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
# add markers to map
for lat, lng, borough, neighborhood in zip(map_data['Latitude'], map_data['Longitude'], map_data['Borough'], map_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
map_toronto

In [19]:
#Let's explore the first neighborhood in our dataframe.
map_data.loc[0, 'Neighborhood']
#Get the neighborhood's latitude and longitude values.
neighborhood_latitude = map_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = map_data.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = map_data.loc[0, 'Neighborhood'] # neighborhood name
borough_name = map_data.loc[0, 'Borough'] 
print('Borough and neighborhood names: {}, {}.'.format(borough_name, neighborhood_name))
print('Latitude and longitude values: {}, {}.'.format(neighborhood_latitude, neighborhood_longitude))

Borough and neighborhood names: Downtown Toronto, Regent Park / Harbourfront.
Latitude and longitude values: 43.6542599, -79.3606359.


In [20]:
#Now, let's get the top 100 venues there within a radius of 500 meters.
radius = 500 

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
print(url) # display URL

results = requests.get(url).json()
#results

https://api.foursquare.com/v2/venues/explore?&client_id=QCPODIZ0VM3OJDR3F4XHQB43T1YVMXP5I3WBALJBPK5K5BPT&client_secret=5LLNK05Z20KKUJD45GXKCC4SSIAW5E0F1WZP0ART4X01BAOI&v=20180604&ll=43.6542599,-79.3606359&radius=500&limit=30


In [21]:
#From the Foursquare lab in the previous module, we know that all the information is in the items key. 
#Before we proceed, let's borrow the get_category_type function from the Foursquare lab.

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']    
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [22]:
#Now we are ready to clean the json and structure it into a pandas dataframe.
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))


30 venues were returned by Foursquare.


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [23]:
#create some functions to replicate the NYC analysis 

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)   
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]



In [24]:
map_venues = getNearbyVenues(names=map_data['Neighborhood'],
                                   latitudes=map_data['Latitude'],
                                   longitudes=map_data['Longitude']
                                  )
map_venues.head()

Regent Park / Harbourfront
Queen's Park / Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond / Adelaide / King
Dufferin / Dovercourt Village
Harbourfront East / Union Station / Toronto Islands
Little Portugal / Trinity
The Danforth West / Riverdale
Toronto Dominion Centre / Design Exchange
Brockton / Parkdale Village / Exhibition Place
India Bazaar / The Beaches West
Commerce Court / Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park / The Junction South
North Toronto West
The Annex / North Midtown / Yorkville
Parkdale / Roncesvalles
Davisville
University of Toronto / Harbord
Runnymede / Swansea
Moore Park / Summerhill East
Kensington Market / Chinatown / Grange Park
Summerhill West / Rathnelly / South Hill / Forest Hill SE / Deer Park
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst  Quay / South Niagara / Island airport
Rosed

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Regent Park / Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Regent Park / Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Regent Park / Harbourfront,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,Regent Park / Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Regent Park / Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [25]:
#Let's check how many venues were returned for each neighborhood
print(map_venues[['Neighborhood', 'Venue']].groupby('Neighborhood').count()) 

#Let's find out how many unique categories can be curated from all the returned venues
print('There are {} unique categories.'.format(len(map_venues['Venue Category'].unique())))

                                                    Venue
Neighborhood                                             
Berczy Park                                            30
Brockton / Parkdale Village / Exhibition Place         22
Business reply mail Processing CentrE                  17
CN Tower / King and Spadina / Railway Lands / H...     18
Central Bay Street                                     30
Christie                                               18
Church and Wellesley                                   30
Commerce Court / Victoria Hotel                        30
Davisville                                             30
Davisville North                                        8
Dufferin / Dovercourt Village                          17
First Canadian Place / Underground city                30
Forest Hill North & West                                4
Garden District, Ryerson                               30
Harbourfront East / Union Station / Toronto Isl...     30
High Park / Th

In [48]:
# one hot encoding
map_onehot = pd.concat([map_venues, pd.get_dummies(map_venues[['Venue Category']], prefix="_", prefix_sep="")], axis=1)

# add neighborhood column back to dataframe
#map_onehot['Neighborhood'] = map_venues['Neighborhood'] 
# move neighborhood column to the first column
#fixed_columns = [map_onehot.columns[-1]] + list(map_onehot.columns[:-1])
#map_onehot = map_onehot[fixed_columns]

map_onehot.drop(['Neighborhood Latitude'],axis=1, inplace=True)
map_onehot.drop(['Neighborhood Longitude'],axis=1, inplace=True)
map_onehot.drop(['Venue Latitude'],axis=1, inplace=True)
map_onehot.drop(['Venue Longitude'],axis=1, inplace=True)
map_onehot.drop(['Venue Category'],axis=1, inplace=True)
map_onehot.drop(['Venue'],axis=1, inplace=True)
map_onehot.head()




,Neighborhood,_Airport,_Airport Food Court,_Airport Gate,_Airport Lounge,_Airport Service,_Airport Terminal,_American Restaurant,_Antique Shop,_Art Gallery,_Arts & Crafts Store,_Asian Restaurant,_Athletics & Sports,_Auto Workshop,_BBQ Joint,_Baby Store,_Bagel Shop,_Bakery,_Bank,_Bar,_Basketball Stadium,_Beer Bar,_Beer Store,_Belgian Restaurant,_Bistro,_Boat or Ferry,_Bookstore,_Boutique,_Breakfast Spot,_Brewery,_Bubble Tea Shop,_Burger Joint,_Burrito Place,_Bus Line,_Butcher,_Café,_Cajun / Creole Restaurant,_Candy Store,_Caribbean Restaurant,_Cheese Shop,_Chinese Restaurant,_Chocolate Shop,_Church,_Climbing Gym,_Clothing Store,_Cocktail Bar,_Coffee Shop,_College Arts Building,_College Auditorium,_College Gym,_College Rec Center,_Colombian Restaurant,_Comfort Food Restaurant,_Comic Shop,_Concert Hall,_Convenience Store,_Cosmetics Shop,_Coworking Space,_Creperie,_Cuban Restaurant,_Dance Studio,_Deli / Bodega,_Department Store,_Dessert Shop,_Diner,_Discount Store,_Distribution Center,_Dog Run,_Donut Shop,_Dumpling Restaurant,_Eastern European Restaurant,_Electronics Store,_Event Space,_Falafel Restaurant,_Farmers Market,_Fast Food Restaurant,_Fish & Chips Shop,_Fish Market,_Flea Market,_Food,_Food & Drink Shop,_Food Court,_Food Truck,_Fountain,_French Restaurant,_Fried Chicken Joint,_Fruit & Vegetable Store,_Furniture / Home Store,_Gaming Cafe,_Garden,_Garden Center,_Gas Station,_Gastropub,_Gay Bar,_General Entertainment,_General Travel,_Gift Shop,_Gluten-free Restaurant,_Gourmet Shop,_Greek Restaurant,_Grocery Store,_Gym,_Gym / Fitness Center,_Harbor / Marina,_Health Food Store,_Historic Site,_History Museum,_Hobby Shop,_Hostel,_Hotel,_IT Services,_Ice Cream Shop,_Indian Restaurant,_Indie Movie Theater,_Intersection,_Italian Restaurant,_Japanese Restaurant,_Jazz Club,_Jewelry Store,_Juice Bar,_Korean Restaurant,_Lake,_Latin American Restaurant,_Lawyer,_Light Rail Station,_Liquor Store,_Lounge,_Market,_Men's Store,_Mexican Restaurant,_Middle Eastern Restaurant,_Miscellaneous Shop,_Modern European Restaurant,_Movie Theater,_Museum,_Music Venue,_Neighborhood,_New American Restaurant,_Nightclub,_Noodle House,_Organic Grocery,_Park,_Performing Arts Venue,_Pet Store,_Pharmacy,_Pizza Place,_Plane,_Playground,_Plaza,_Pub,_Ramen Restaurant,_Record Shop,_Rental Car Location,_Restaurant,_Salad Place,_Salon / Barbershop,_Sandwich Place,_Sculpture Garden,_Seafood Restaurant,_Shopping Mall,_Skate Park,_Skating Rink,_Smoke Shop,_Spa,_Speakeasy,_Sporting Goods Shop,_Sports Bar,_Stadium,_Stationery Store,_Steakhouse,_Supermarket,_Sushi Restaurant,_Swim School,_Tailor Shop,_Taiwanese Restaurant,_Tea Room,_Tennis Court,_Thai Restaurant,_Theater,_Theme Restaurant,_Toy / Game Store,_Trail,_Train Station,_Vegetarian / Vegan Restaurant,_Video Game Store,_Vietnamese Restaurant,_Wine Bar,_Wine Shop,_Yoga Studio
0,Regent Park / Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Regent Park / Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Regent Park / Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [50]:
#Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
map_grouped = map_onehot.groupby('Neighborhood').mean().reset_index()
map_grouped
map_grouped.head()


,Neighborhood,_Airport,_Airport Food Court,_Airport Gate,_Airport Lounge,_Airport Service,_Airport Terminal,_American Restaurant,_Antique Shop,_Art Gallery,_Arts & Crafts Store,_Asian Restaurant,_Athletics & Sports,_Auto Workshop,_BBQ Joint,_Baby Store,_Bagel Shop,_Bakery,_Bank,_Bar,_Basketball Stadium,_Beer Bar,_Beer Store,_Belgian Restaurant,_Bistro,_Boat or Ferry,_Bookstore,_Boutique,_Breakfast Spot,_Brewery,_Bubble Tea Shop,_Burger Joint,_Burrito Place,_Bus Line,_Butcher,_Café,_Cajun / Creole Restaurant,_Candy Store,_Caribbean Restaurant,_Cheese Shop,_Chinese Restaurant,_Chocolate Shop,_Church,_Climbing Gym,_Clothing Store,_Cocktail Bar,_Coffee Shop,_College Arts Building,_College Auditorium,_College Gym,_College Rec Center,_Colombian Restaurant,_Comfort Food Restaurant,_Comic Shop,_Concert Hall,_Convenience Store,_Cosmetics Shop,_Coworking Space,_Creperie,_Cuban Restaurant,_Dance Studio,_Deli / Bodega,_Department Store,_Dessert Shop,_Diner,_Discount Store,_Distribution Center,_Dog Run,_Donut Shop,_Dumpling Restaurant,_Eastern European Restaurant,_Electronics Store,_Event Space,_Falafel Restaurant,_Farmers Market,_Fast Food Restaurant,_Fish & Chips Shop,_Fish Market,_Flea Market,_Food,_Food & Drink Shop,_Food Court,_Food Truck,_Fountain,_French Restaurant,_Fried Chicken Joint,_Fruit & Vegetable Store,_Furniture / Home Store,_Gaming Cafe,_Garden,_Garden Center,_Gas Station,_Gastropub,_Gay Bar,_General Entertainment,_General Travel,_Gift Shop,_Gluten-free Restaurant,_Gourmet Shop,_Greek Restaurant,_Grocery Store,_Gym,_Gym / Fitness Center,_Harbor / Marina,_Health Food Store,_Historic Site,_History Museum,_Hobby Shop,_Hostel,_Hotel,_IT Services,_Ice Cream Shop,_Indian Restaurant,_Indie Movie Theater,_Intersection,_Italian Restaurant,_Japanese Restaurant,_Jazz Club,_Jewelry Store,_Juice Bar,_Korean Restaurant,_Lake,_Latin American Restaurant,_Lawyer,_Light Rail Station,_Liquor Store,_Lounge,_Market,_Men's Store,_Mexican Restaurant,_Middle Eastern Restaurant,_Miscellaneous Shop,_Modern European Restaurant,_Movie Theater,_Museum,_Music Venue,_Neighborhood,_New American Restaurant,_Nightclub,_Noodle House,_Organic Grocery,_Park,_Performing Arts Venue,_Pet Store,_Pharmacy,_Pizza Place,_Plane,_Playground,_Plaza,_Pub,_Ramen Restaurant,_Record Shop,_Rental Car Location,_Restaurant,_Salad Place,_Salon / Barbershop,_Sandwich Place,_Sculpture Garden,_Seafood Restaurant,_Shopping Mall,_Skate Park,_Skating Rink,_Smoke Shop,_Spa,_Speakeasy,_Sporting Goods Shop,_Sports Bar,_Stadium,_Stationery Store,_Steakhouse,_Supermarket,_Sushi Restaurant,_Swim School,_Tailor Shop,_Taiwanese Restaurant,_Tea Room,_Tennis Court,_Thai Restaurant,_Theater,_Theme Restaurant,_Toy / Game Store,_Trail,_Train Station,_Vegetarian / Vegan Restaurant,_Video Game Store,_Vietnamese Restaurant,_Wine Bar,_Wine Shop,_Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.033333,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.066667,0.0,0.000000,0.033333,0.066667,0.0,0.0,0.033333,0.000000,0.0,0.000000,0.033333,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.066667,0.0,0.0,0.0,0.033333,0.000000,0.0,0.0,0.000000,0.0,0.066667,0.100000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.033333,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.066667,0.000000,0.0,0.033333,0.0,0.0,0.0,0.0,0.0,0.033333,0.033333,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.033333,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.033333,0.0,0.0,0.0,0.000000,0.0,0.0,0.033333,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.033333,0.0,0.0,0.000000,0.000000,0.066667,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.0,0.000000
1,Brockton / Parkdale Village / E

In [52]:
#Let's print each neighborhood along with the top 5 most common venues
num_top_venues = 5
for hood in map_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = map_grouped[map_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                 venue  freq
0         _Coffee Shop  0.10
1              _Bakery  0.07
2      _Farmers Market  0.07
3  _Seafood Restaurant  0.07
4            _Beer Bar  0.07


----Brockton / Parkdale Village / Exhibition Place----
             venue  freq
0            _Café  0.14
1  _Breakfast Spot  0.09
2     _Coffee Shop  0.09
3    _Intersection  0.05
4       _Pet Store  0.05


----Business reply mail Processing CentrE----
                   venue  freq
0                   _Spa  0.06
1  _Fast Food Restaurant  0.06
2            _Comic Shop  0.06
3           _Pizza Place  0.06
4            _Restaurant  0.06


----CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst  Quay / South Niagara / Island airport----
                  venue  freq
0      _Airport Service  0.17
1       _Airport Lounge  0.11
2     _Airport Terminal  0.11
3              _Airport  0.06
4  _Rental Car Location  0.06


----Central Bay Street----
                 venue  freq
0  

In [53]:
#Now let's create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = map_grouped['Neighborhood']
for ind in np.arange(map_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(map_grouped.iloc[ind, :], num_top_venues)
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,_Coffee Shop,_Seafood Restaurant,_Farmers Market,_Café,_Bakery,_Beer Bar,_Cocktail Bar,_Bistro,_Restaurant,_Fish Market
1,Brockton / Parkdale Village / Exhibition Place,_Café,_Breakfast Spot,_Coffee Shop,_Intersection,_Performing Arts Venue,_Italian Restaurant,_Restaurant,_Stadium,_Bar,_Bakery
2,Business reply mail Processing CentrE,_Garden,_Gym / Fitness Center,_Skate Park,_Smoke Shop,_Spa,_Restaurant,_Burrito Place,_Butcher,_Farmers Market,_Fast Food Restaurant
3,CN Tower / King and Spadina / Railway Lands / ...,_Airport Service,_Airport Lounge,_Airport Terminal,_Airport,_Harbor / Marina,_Coffee Shop,_Plane,_Rental Car Location,_Sculpture Garden,_Boat or Ferry
4,Central Bay Street,_Coffee Shop,_Italian Restaurant,_Spa,_Café,_Yoga Studio,_Modern European Restaurant,_Ice Cream Shop,_Dessert Shop,_Sandwich Place,_Bubble Tea Shop


In [55]:
#Run k-means to cluster the neighborhood into 5 clusters.
# set number of clusters
kclusters = 5
map_grouped_clustering = map_grouped.drop('Neighborhood', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(map_grouped_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

#Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
map_merged = map_data
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
map_merged = map_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
map_merged.head() # check the last columns!

,index,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,4,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636,0,_Coffee Shop,_Bakery,_Park,_Breakfast Spot,_Café,_Mexican Restaurant,_Historic Site,_Restaurant,_Farmers Market,_Dessert Shop
1,6,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494,0,_Coffee Shop,_Diner,_Fried Chicken Joint,_Mexican Restaurant,_Sandwich Place,_Burger Joint,_Burrito Place,_Café,_Park,_College Auditorium
2,13,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,_Coffee Shop,_Clothing Store,_Café,_Hotel,_Plaza,_Shopping Mall,_Sandwich Place,_Diner,_Restaurant,_Ramen Restaurant
3,22,Downtown Toronto,St. James Town,43.651494,-79.375418,0,_Café,_Coffee Shop,_Hotel,_Restaurant,_Japanese Restaurant,_Gastropub,_Gym,_Middle Eastern Restaurant,_Diner,_Beer Bar
4,30,East Toronto,The Beaches,43.676357,-79.293031,4,_Neighborhood,_Trail,_Pub,_Health Food Store,_Yoga Studio,_Creperie,_Distribution Center,_Discount Store,_Diner,_Dessert Shop


In [57]:
#Finally, let's visualize the resulting clusters
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(map_merged['Latitude'], map_merged['Longitude'], map_merged['Neighborhood'], map_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)   
map_clusters